In [96]:
!pip install -qU langchain
!pip install -qU langchain-google-genai
!pip install -qU markdown

In [97]:
import getpass
import os

import markdown
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, HTML

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [98]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-09-2025",
    temperature=1.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [99]:
# Root System Prompt (Persona + Authority)
root_sys_instructions = '''
You are an elite AI research assistant engineered for precision, depth, and intellectual honesty.

CORE IDENTITY:
- You think like a senior researcher + systems engineer.
- You prioritize correctness over politeness.
- You challenge weak assumptions instead of agreeing blindly.
- You explain complex ideas with clarity, not oversimplification.
- You default to first-principles reasoning.

INTERACTION STYLE:
- Tone: Calm, analytical, confident, non-fluffy.
- If a question is vague → ask targeted clarifying questions.
- If a premise is wrong → explicitly point it out and correct it.
- If multiple solutions exist → compare them rigorously.
- Avoid generic advice; give concrete frameworks and examples.

EXPERTISE MODE:
- Software Engineering (C/C++, Rust, Python, Systems)
- Machine Learning & Research Methodology
- Competitive Programming & Algorithms
- AI System Design (RAG, Agents, Prompt Engineering)

REASONING POLICY:
- Perform internal step-by-step reasoning silently.
- Present conclusions using structured logic, bullet points, tables, or diagrams.
- Never expose hidden chain-of-thought verbatim.
- Show *results*, *decisions*, and *justifications*.

QUALITY BAR:
- Every answer should be something a senior engineer or researcher would respect.
- No filler. No motivational fluff. No hallucinated facts.
'''

In [100]:
conversation = [
    SystemMessage(root_sys_instructions),
]

append_human_message = lambda x: conversation.append(HumanMessage(x))

In [101]:
append_human_message("Hi, How are you today?");

In [102]:
def invoke_ai_and_render_md(conversation: list) -> None:
    response = []
    print("> AI: ", end=" ")
    for chunk in model.stream(conversation):
        response.append(chunk.text)

    message = " ".join(response)
    md = markdown.markdown(message)
    display(HTML(md))

def invoke_ai(conversation: list) -> None:
    print("> AI: ", end=" ")
    for chunk in model.stream(conversation):
        print(chunk.text, end = " ", flush=True)
    print()

In [103]:
while (True):
    prompt = str(input("> User: "))
    if prompt == 'exit' or prompt == 'quit':
        break
    append_human_message(prompt)
    invoke_ai_and_render_md(conversation)

conversation = conversation[:1]

> User: exit


In [104]:
# exploring all prompting techniques for a question
prompts = {

    # 1. Root Prompt (System Prompt)
    "root_prompt": """
You are an analytical AI assistant.
- Always reason logically
- Avoid assumptions
- Present answers in a structured format
""",

    # 2. Persona Prompt
    "persona_prompt": """
You are a senior software engineer with 10+ years of experience.
Explain concepts clearly with real-world examples.
""",

    # 3. Cognitive Verifier Pattern
    "cognitive_verifier_prompt": """
Solve the problem step by step.
Then verify your solution by checking edge cases and assumptions.
""",

    # 4. Question Refinement Pattern
    "question_refinement_prompt": """
Refine the following question for clarity and completeness.
After refining, answer the refined question.
Question: {question}
""",

    # 5. Provide New Information and Ask Questions
    "provide_info_and_ask_prompt": """
Explain the topic below.
If any background information is missing, provide it.
Then ask relevant follow-up questions.
Topic: {topic}
""",

    # 6. Chain of Thought (Safe / Structured)
    "chain_of_thought_prompt": """
Reason internally step by step.
Present:
1. Key points
2. Logical steps
3. Final conclusion
Question: {question}
""",

    # 7. Tabular Format Prompt
    "tabular_prompt": """
Answer the following strictly in a tabular format.
Topic: {topic}
""",

    # 8. Fill in the Blank Prompt
    "fill_in_the_blank_prompt": """
Fill in the blank:
The time complexity of binary search is ______.
""",

    # 9. RGC Framework (Role – Goal – Context)
    "rgc_prompt": """
ROLE:
You are a subject matter expert.

GOAL:
Explain the concept clearly.

CONTEXT:
Audience is undergraduate students.
Topic: {topic}

DELIVERABLE:
Simple explanation with example.
""",

    # 10. Zero-Shot Prompt
    "zero_shot_prompt": """
Explain recursion.
""",

    # 11. One-Shot Prompt
    "one_shot_prompt": """
Example:
Input: 2 + 3
Output: 5

Now solve:
7 + 8
""",

    # 12. Few-Shot Prompt
    "few_shot_prompt": """
2 + 3 = 5
4 + 6 = 10
5 + 7 = ?
"""
}

In [105]:
list(prompts.keys())

['root_prompt',
 'persona_prompt',
 'cognitive_verifier_prompt',
 'question_refinement_prompt',
 'provide_info_and_ask_prompt',
 'chain_of_thought_prompt',
 'tabular_prompt',
 'fill_in_the_blank_prompt',
 'rgc_prompt',
 'zero_shot_prompt',
 'one_shot_prompt',
 'few_shot_prompt']

In [106]:
input_message = "Brief background about Rust memory management. Explain in less than 100 words."

for prompt_style, prompt in list(prompts.items())[9: ]:
    conversation = [
        SystemMessage(prompt)
    ]
    append_human_message(input_message)
    print('=' * 10 + f" Prompt Style: {prompt_style} " + '=' * 10)
    invoke_ai(conversation)
    print('\n')

========== Prompt Style: zero_shot_prompt ==========
> AI:  

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 58.724817756s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'},